## Importing the libraries

In [1]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense

from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

2023-08-16 02:09:05.352142: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Importing the dataset

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values
print(dataset.shape, X.shape, y.shape)

(10000, 14) (10000, 10) (10000,)


In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
label_encoder_X = LabelEncoder()
X[:, 2] = label_encoder_X.fit_transform(X[:, 2])

ct = ColumnTransformer(
    transformers=[("Geo", OneHotEncoder(), [1])],
    remainder="passthrough",
)
X = ct.fit_transform(X)
X = X[:, 1:]

## Define a create function for model

In [5]:
def create_model():
    model = Sequential()
    model.add(InputLayer(input_shape=(11, ), name='input_layer'))
    model.add(Dense(units=6, activation='relu', name='hidden_layer_1'))
    model.add(Dense(units=1, activation='sigmoid', name='output_layer'))
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

## Define the K-Fold Cross-Validation Split

In [7]:
kfold = KFold(n_splits=4, shuffle=True, random_state=0)
accuracies = []
for train_index, test_index in kfold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    model = create_model()
    model.fit(X_train, y_train, batch_size=100, epochs=20)

    y_pred = model.predict(X_test)
    y_pred = y_pred > 0.5

    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    print(f'Accuracy: {accuracy}')

Epoch 1/20
75/75 [==============================] - 1s 4ms/step - loss: 0.5889 - accuracy: 0.7232
Epoch 2/20
75/75 [==============================] - 0s 5ms/step - loss: 0.5209 - accuracy: 0.7780
Epoch 3/20
75/75 [==============================] - 0s 4ms/step - loss: 0.4816 - accuracy: 0.7991
Epoch 4/20
75/75 [==============================] - 0s 5ms/step - loss: 0.4581 - accuracy: 0.8096
Epoch 5/20
75/75 [==============================] - 0s 3ms/step - loss: 0.4437 - accuracy: 0.8148
Epoch 6/20
75/75 [==============================] - 0s 3ms/step - loss: 0.4339 - accuracy: 0.8179
Epoch 7/20
75/75 [==============================] - 0s 4ms/step - loss: 0.4273 - accuracy: 0.8183
Epoch 8/20
75/75 [==============================] - 0s 6ms/step - loss: 0.4221 - accuracy: 0.8208
Epoch 9/20
75/75 [==============================] - 0s 6ms/step - loss: 0.4179 - accuracy: 0.8219
Epoch 10/20
75/75 [==============================] - 0s 6ms/step - loss: 0.4143 - accuracy: 0.8223
Epoch 11/20
75/75 [

In [14]:
mean = np.array(accuracies).mean()
std = np.array(accuracies).std()
print(f"model accuracy with mean:{mean:.3f} and std: {std:.3f}")

model accuracy with mean:0.833 and std: 0.009
